In [485]:
# import libs
import urllib
import numpy as np
import re
import pandas as pd
import warnings

from tqdm import tqdm

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [103]:
reg_1 = r"(?:<td><b>)(.+?)(?:<\/b>.+?)([\d,]+?)(?:<)"
reg_2 = r"(?:<b>)([^=]+?)(?:\:<\/b>.+?>)(.+?)(?:<\/td>)"
reg_3 = r"(?:cast_new.+?<h1>)(.+?)(?:</h1>.+?)(.+?)(?:</div>)"

In [121]:
save = pd.read_csv('name_and_links.csv')

In [139]:
def extract(link, name):
    tempurl = "https://www.the-numbers.com/movie/" + link
    page = urllib.request.Request(tempurl, headers={'User-Agent' : "Magic Browser"})
    page = urllib.request.urlopen(page)
    # read page
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")

    # subset
    start_index = html.find('\"main\"')
    end_index = html.find('Further financial d')
    financial = html[start_index:end_index]
    
    start_index = html.find('Movie Details')
    end_index = html.find('cast-and-crew')
    detail = html[start_index:end_index]
    
    start_index = html.find('cast-and-crew\"')
    end_index = html.find('id=\"news\"')
    cast = html[start_index:end_index]
    
    p_financial = re.findall(reg_1, financial.replace("\n", ""), re.MULTILINE)
    p_detail = re.findall(reg_2, detail.replace("\n", ""), re.MULTILINE)
    p_cast = re.findall(reg_3, cast.replace("\n", ""), re.MULTILINE)
    p = [("link", link), ("name", name)] + p_financial + p_detail + p_cast
    
    temp = pd.DataFrame(p, columns=('key', name))
    
    return temp

In [207]:
df = save[1:10]

In [200]:
# for index, row in tqdm(df.iterrows(), total = df.shape[0]):
#     name = row['moviename']
#     link = row['link']
    
#     temp = extract(link, name)
#     if (index == 0):
#         raw = temp
#     else:
#         raw = raw.set_index('key').join(temp.set_index('key'), how = "outer", lsuffix = '_left', rsuffix = '_right')
#         raw['key'] = raw.axes[0].tolist()

  0%|          | 0/99 [00:00<?, ?it/s]C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\478875361.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  raw['key'] = raw.axes[0].tolist()
  1%|          | 1/99 [00:02<03:31,  2.16s/it]C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\478875361.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  raw['key'] = raw.axes[0].tolist()
  2%|▏         | 2/99 [00:03<03:07,  1.93s/it]C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\478875361.py:10: PerformanceWarning: DataFrame is highly fra

In [216]:
tqdm.pandas()

In [290]:
def get(df):
    print("Started working on row " + str(df.index[0]))
    temp = df.progress_apply(lambda x: extract(x['link'], x['moviename']), axis = 1)
    # temp = [d.set_index('key') for d in temp]
    # temp = temp[0].join(temp[1:])
    return temp

In [292]:
# save_split = np.array_split(save, 2)
# res = [get(d) for d in save_split]

In [307]:
res = get(save)

Started working on row 0


  0%|          | 2/6383 [00:03<2:47:28,  1.57s/it]


KeyboardInterrupt: 

In [308]:
df = res

In [316]:
backup = res

In [309]:
def f1(df):
    df['key'] = df.axes[0].tolist()
    return df

In [310]:
df = [d.set_index('key') for d in df]
# df = [f1(d) for d in df]

In [381]:
def f2(df):
    df.columns = [df.loc['link'][0]]
    return df

In [382]:
df = [f2(d) for d in df]

In [387]:
rawdata = df[0].join(df[1:], how = "outer")

In [392]:
raw = rawdata.transpose()

In [394]:
raw.to_csv('raw.csv')

In [395]:
##################################################################################################################
# Extract data
##################################################################################################################

In [399]:
df = raw

In [414]:
link_and_name = df[df.columns[0:2]]

In [426]:
dom_box = df[df.columns[2]].to_frame()
dom_box = dom_box.replace(',', '', regex = True)
dom_box = dom_box.apply(pd.to_numeric)

In [427]:
int_box = df[df.columns[3]].to_frame()
int_box = int_box.replace(',', '', regex = True)
int_box = int_box.apply(pd.to_numeric)

In [434]:
wor_box = df[df.columns[4]].to_frame()
wor_box = wor_box.replace(',', '', regex = True)
wor_box = wor_box.apply(pd.to_numeric)

In [546]:
dom_rel = pd.Series([])
year = pd.Series([])
reg_r1 = r"(?:<td>)(.*?,{0,1} {0,1}\d{4})"
reg_r2 = r"\d{4}"
for index, row in df[df.columns[5]].to_frame().iterrows():
    d = ""
    d2 = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
        d2 = re.findall(reg_r2, d[0])
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        dom_rel = dom_rel.append(pd.Series(data = d, index = [index]))
        year = year.append(pd.Series(data = d2, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\426625156.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dom_rel = pd.Series([])
C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\426625156.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  year = pd.Series([])


In [556]:
rel = pd.DataFrame({'key':dom_rel.index, 'Domestic Release':dom_rel.values, 'Dom Year':year.values})
rel = rel.set_index('key')

In [558]:
int_rel = pd.Series([])
int_year = pd.Series([])
reg_r1 = r"(?:<td>)(.*?,{0,1} {0,1}\d{4})"
reg_r2 = r"\d{4}"
for index, row in df[df.columns[6]].to_frame().iterrows():
    d = ""
    d2 = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
        d2 = re.findall(reg_r2, d[0])
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        int_rel = int_rel.append(pd.Series(data = d, index = [index]))
        int_year = int_year.append(pd.Series(data = d2, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\3760324377.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  int_rel = pd.Series([])
C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\3760324377.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  int_year = pd.Series([])


In [559]:
int_rel = pd.DataFrame({'key':int_rel.index, 'International Release':int_rel.values, 'Int Year':int_year.values})
int_rel = int_rel.set_index('key')

In [580]:
rat = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/)"
for index, row in df[df.columns[7]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
        d = d[0]
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        rat = rat.append(pd.Series(data = d, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\1044994531.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  rat = pd.Series([])


In [581]:
rat = rat.to_frame()
rat.columns = ['Rating']

In [596]:
run = pd.Series([])
reg_r1 = r"(\d+?)(?: minutes)"
for index, row in df[df.columns[8]].to_frame().iterrows():
    d = np.nan
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
        d = d[0]
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        run = run.append(pd.Series(data = d, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\1515363466.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  run = pd.Series([])


In [597]:
run = run.to_frame()
run.columns = ['Runtime']
run = pd.to_numeric(run['Runtime'])

In [607]:
fra = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/)"
for index, row in df[df.columns[9]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        fra = fra.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\503250315.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fra = pd.Series([])


In [610]:
fra = fra.to_frame()
fra.columns = ['Franchise']

In [624]:
keyw = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[11]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        keyw = keyw.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2161588769.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  keyw = pd.Series([])


In [625]:
keyw = keyw.to_frame()
keyw.columns = ['Keywords']

In [633]:
gen = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[13]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        gen = gen.append(pd.Series(data = d, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2265188621.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  gen = pd.Series([])


In [642]:
gen = gen.to_frame()
gen.columns = ['Genre']

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [651]:
pro = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[14]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        pro = pro.append(pd.Series(data = d, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\1207845012.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pro = pd.Series([])


In [652]:
pro = pro.to_frame()
pro.columns = ['Production Method']

In [659]:
cre = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[15]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        cre = cre.append(pd.Series(data = d, index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2440901688.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cre = pd.Series([])


In [660]:
cre = cre.to_frame()
cre.columns = ['Creative Type']

In [665]:
com = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[16]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        com = com.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\3104245943.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  com = pd.Series([])


In [666]:
com = com.to_frame()
com.columns = ['Production/Financing Companies']

In [672]:
cou = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[17]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        cou = cou.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\4123167272.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cou = pd.Series([])


In [673]:
cou = cou.to_frame()
cou.columns = ['Production Country']

In [677]:
lan = pd.Series([])
reg_r1 = r"(?:\">)(.+?)(?:<\/a)"
for index, row in df[df.columns[18]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        lan = lan.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\4183855760.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lan = pd.Series([])


In [678]:
lan = lan.to_frame()
lan.columns = ['Languages']

In [682]:
lead = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[19]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        lead = lead.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2397918207.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lead = pd.Series([])


In [683]:
lead = lead.to_frame()
lead.columns = ['Leading Cast']

In [690]:
sup = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[20]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        sup = sup.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\3410085375.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sup = pd.Series([])


In [691]:
sup = sup.to_frame()
sup.columns = ['Supporting Cast']

In [779]:
tec = pd.Series([])
dirc = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
for index, row in df[df.columns[21]].to_frame().iterrows():
    d = ""
    d2 = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
        d2 = re.findall(reg_r2, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        tec = tec.append(pd.Series(data = [d], index = [index]))
        dirc = dirc.append(pd.Series(data = [d2], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\334820549.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tec = pd.Series([])
C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\334820549.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dirc = pd.Series([])


In [780]:
tec = tec.to_frame()
tec.columns = ['Production and Technical Credits']

In [781]:
dirc = dirc.to_frame()
dirc.columns = ['Director']

In [708]:
est_dom_dvd = df[df.columns[22]].to_frame()
est_dom_dvd = est_dom_dvd.replace(',', '', regex = True)
est_dom_dvd = est_dom_dvd.apply(pd.to_numeric)

In [712]:
est_dom_blue = df[df.columns[23]].to_frame()
est_dom_blue = est_dom_blue.replace(',', '', regex = True)
est_dom_blue = est_dom_blue.apply(pd.to_numeric)

In [715]:
est_dom_tol = df[df.columns[24]].to_frame()
est_dom_tol = est_dom_tol.replace(',', '', regex = True)
est_dom_tol = est_dom_tol.apply(pd.to_numeric)

In [718]:
ens = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[25]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        ens = ens.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2377893270.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ens = pd.Series([])


In [719]:
ens = ens.to_frame()
ens.columns = ['Lead Ensemble Members']

In [724]:
cam = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[26]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        cam = cam.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2700461392.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cam = pd.Series([])


In [725]:
cam = cam.to_frame()
cam.columns = ['Cameos']

In [863]:
t = df[df.columns[27]].to_frame().iloc[:, 0][1]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [864]:
tec.iloc[1] = [d]
dirc.iloc[1] = [d2]

In [861]:
ind = df[df[df.columns[28]].notnull()].index
t = df[df.columns[28]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [862]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [805]:
unc = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[29]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        unc = unc.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\1981062566.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  unc = pd.Series([])


In [806]:
unc = unc.to_frame()
unc.columns = ['Uncategorized Crew']

In [810]:
nar = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[30]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        nar = nar.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\4025616791.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nar = pd.Series([])


In [811]:
nar = nar.to_frame()
nar.columns = ['Narrator(s)']

In [859]:
ind = df[df[df.columns[31]].notnull()].index
t = df[df.columns[31]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [860]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [828]:
ext = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[32]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        ext = ext.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\2512667907.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ext = pd.Series([])


In [829]:
ext = ext.to_frame()
ext.columns = ['Extras']

In [857]:
ind = df[df[df.columns[33]].notnull()].index
t = df[df.columns[33]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [858]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [867]:
ind = df[df[df.columns[34]].notnull()].index
t = df[df.columns[34]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
d = ""
d = re.findall(reg_r1, t, re.MULTILINE)

In [868]:
unc.loc[ind].iloc[0, 0] = [d]

In [855]:
ind = df[df[df.columns[35]].notnull()].index
t = df[df.columns[35]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [856]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [853]:
ind = df[df[df.columns[36]].notnull()].index
t = df[df.columns[36]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [854]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [847]:
doc_sub = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[37]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        doc_sub = doc_sub.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\1114625216.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  doc_sub = pd.Series([])


In [848]:
doc_sub = doc_sub.to_frame()
doc_sub.columns = ['Documentary Subject(s)']

In [851]:
ind = df[df[df.columns[38]].notnull()].index
t = df[df.columns[38]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [852]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [872]:
ind = df[df[df.columns[39]].notnull()].index
t = df[df.columns[39]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [873]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [877]:
interviewee = pd.Series([])
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
for index, row in df[df.columns[37]].to_frame().iterrows():
    d = ""
    if not pd.isna(row[0]):
        d = re.findall(reg_r1, row[0], re.MULTILINE)
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        interviewee = interviewee.append(pd.Series(data = [d], index = [index]))

C:\Users\kongw\AppData\Local\Temp\ipykernel_29100\1385328225.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  interviewee = pd.Series([])


In [878]:
interviewee = interviewee.to_frame()
interviewee.columns = ['Interviewee(s)']

In [881]:
ind = df[df[df.columns[41]].notnull()].index
t = df[df.columns[41]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [882]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [884]:
ind = df[df[df.columns[42]].notnull()].index
t = df[df.columns[42]].to_frame().loc[ind].iloc[:,0][0]
reg_r1 = r"(?:\">)([^<>]+?)(?:<\/a)"
reg_r2 = r"(?:\">)([^<>]+?)(?:<.+?>Director<)"
d = ""
d2 = ""
d = re.findall(reg_r1, t, re.MULTILINE)
d2 = re.findall(reg_r2, t, re.MULTILINE)

In [885]:
tec.loc[ind].iloc[0, 0] = [d]
dirc.loc[ind].iloc[0, 0] = [d2]

In [893]:
cleaned = [link_and_name, dom_box, int_box, wor_box, rel, int_rel, rat, run, fra, keyw, gen, pro, cre, com, cou, 
           lan, lead, sup, tec, dirc, est_dom_dvd, est_dom_blue, est_dom_tol, ens, cam, unc, nar, ext, doc_sub, interviewee]
data = cleaned[0].join(cleaned[1:], how = "outer")

In [891]:
# i = 42
# print(df[df.columns[i]].to_frame().head(3))
# print(df[df.columns[i]].to_frame().iloc[:, 0][0])

In [895]:
#####################################################################################################################
# more cleaning

In [896]:
numbers = pd.read_csv('numbers.csv')

In [971]:
dataset = pd.concat([numbers.reset_index(drop=True), data.reset_index(drop=True)], axis=1)

In [972]:
# fix merging datasets
good = dataset[dataset.iloc[:, 1] == dataset.iloc[:, 6]]
bad = dataset[dataset.iloc[:, 1] != dataset.iloc[:, 6]]

df_1 = bad.iloc[:, range(1, 3)]
df_2 = bad.iloc[:, 5:]

df_1 = df_1.drop(index = [4834])

temp = pd.concat([df_1.reset_index(drop=True), df_2.reset_index(drop=True)], axis=1)
dataset = pd.concat([temp, good])

In [974]:
good = dataset[dataset.iloc[:, 0] == dataset.iloc[:, 3]]
bad = dataset[dataset.iloc[:, 0] != dataset.iloc[:, 3]]

In [976]:
# fix merging datasets
dataset = dataset.drop(index = [1525])
good = dataset[dataset.iloc[:, 0] == dataset.iloc[:, 3]]
bad = dataset[dataset.iloc[:, 0] != dataset.iloc[:, 3]]

df_1 = bad.iloc[:, range(0, 2)]
df_2 = bad.iloc[:, 2:]

bad = df_1.join(df_2.set_index('name'), on = 'moviename')
dataset = pd.concat([bad, good])

In [978]:
dataset = dataset.drop(['name', 'number', 'Domestic', 'Worldwide'], axis = 1)

In [983]:
dataset.to_csv('metadata.csv', index = False)